In [18]:
import pandas as pd
import requests
def dfToDict(df): 
    return df.T.to_dict().values()
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data'
df = pd.read_csv(url,header=None)
df.columns = ['buying','maint','doors','persons','lug_boot','safety','class']

In [19]:
data = dfToDict(df[['buying','maint','doors','persons','lug_boot','safety']])

In [41]:
from sklearn.feature_extraction import DictVectorizer
vec = DictVectorizer()
X = vec.fit_transform(data).toarray()
print vec.get_feature_names()
s = pd.Series(df['class'], dtype="category")
Y = s.cat.rename_categories([1,2,3,4]).as_matrix()

['buying=high', 'buying=low', 'buying=med', 'buying=vhigh', 'doors=2', 'doors=3', 'doors=4', 'doors=5more', 'lug_boot=big', 'lug_boot=med', 'lug_boot=small', 'maint=high', 'maint=low', 'maint=med', 'maint=vhigh', 'persons=2', 'persons=4', 'persons=more', 'safety=high', 'safety=low', 'safety=med']


In [43]:
from sklearn import cross_validation
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, Y, test_size=0.4, random_state=0)

In [44]:
X_train.shape, y_train.shape

((1036L, 21L), (1036L,))

In [45]:
X_test.shape, y_test.shape

((692L, 21L), (692L,))

In [56]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(max_depth=6, n_estimators=12, max_features=1)

In [59]:
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)
print score

0.734104046243


In [60]:
from sklearn.cluster import KMeans
kmeans = KMeans(init='k-means++', n_clusters=4, n_init=10)

In [61]:
kmeans.fit(X)

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=4, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

In [62]:
Z = kmeans.predict(X)

In [63]:
Z

array([1, 1, 1, ..., 3, 2, 2])

In [64]:
from sklearn.metrics import mean_squared_error

In [66]:
df1 = df

In [68]:
df1['cluster'] = Z

In [70]:
data2 = dfToDict(df[['buying','maint','doors','persons','lug_boot','safety','cluster']])

In [71]:
vec = DictVectorizer()
X = vec.fit_transform(data2).toarray()
print vec.get_feature_names()
s = pd.Series(df1['class'], dtype="category")
Y = s.cat.rename_categories([1,2,3,4]).as_matrix()

['buying=high', 'buying=low', 'buying=med', 'buying=vhigh', 'cluster', 'doors=2', 'doors=3', 'doors=4', 'doors=5more', 'lug_boot=big', 'lug_boot=med', 'lug_boot=small', 'maint=high', 'maint=low', 'maint=med', 'maint=vhigh', 'persons=2', 'persons=4', 'persons=more', 'safety=high', 'safety=low', 'safety=med']


In [72]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, Y, test_size=0.4, random_state=0)

In [81]:
classifier = RandomForestClassifier(max_depth=6, n_estimators=12, max_features=1)
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)
print score

0.745664739884


In [80]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators=100)
clf.fit(X_train, y_train)
score = clf.score(X_test, y_test)
print score

0.842485549133


In [83]:
from sklearn.ensemble import GradientBoostingClassifier
clf2 = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(X_train, y_train)
clf2.fit(X_train, y_train)
score = clf2.score(X_test, y_test)       
print score

0.890173410405


In [86]:
from sklearn.metrics import mean_squared_error
print mean_squared_error(y_test,clf2.predict(X_test))

0.371387283237


In [88]:
df2 = pd.DataFrame()

In [89]:
df2['true'] = y_test
df2['predicted'] = clf2.predict(X_test)

In [91]:
df2.to_clipboard()